# Building vocabulary from Wikipedia articles

Pre-processing steps we take:
1. All text is lowercases
2. We replace '-' with space because '-' containing words most likely are composed of words already present in the vocabulary
3. We repalce numbers and digits with '#NUMBER' to avoid unneccessary increase in the vocabulary size
4. We replace the hyperlinks by '#HLINK' because for our our study purposes hyperlinks are not useful

Building vocabulary:
1. Initially we were using NLTK's vocabulary for English that contains approximately 230k words. While working with the Wikipedia articles we realized that this 230k vocabulary is not enough. To give an example, 'marxism' word was not present in the NLTK's vocabulary. Now, we may think that if we are filtering sentences based on words spoken by siz year olds then 'marxism', in particular, might not be that important. But, we don't know what is missing in the NLTK's vocabulary and we don't want to miss out on selecting a simple sentence because the word was not present in NLTK.
2. We first sequentially go through wikipedia articles building the vocabulary, until the vocabulary size reaches 2 million. There are many information resources that indicate actual vocabulary size of English is much smaller. But, 2 million is just to be safe. It required 260k articles to develop a vocabulary of 2 million.
3. We check all words from NLTK and all words from AOCHildes are present in the 2 million
4. We sample 260k documents from previously unseen document, build new vocabulary and merge with the existing vocabulary

In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import json
import os
import glob
from datasets import load_dataset
from tqdm import tqdm
#from english_words import english_words_set
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zipf
from scipy.optimize import curve_fit
from nltk.corpus import words
import vocab_utils as utils_
from nltk import FreqDist, word_tokenize, wordpunct_tokenize
import re

In [3]:
#
nltk_vocab = words.words()
aoc_vocab = pd.read_csv('CHILDES_vocab_age.csv').loc[:, 'word'].unique().tolist()

LookupError: 
**********************************************************************
  Resource [93mwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('words')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/words[0m

  Searched in:
    - '/home/vdeshpande/nltk_data'
    - '/home/vdeshpande/miniconda3/envs/general/nltk_data'
    - '/home/vdeshpande/miniconda3/envs/general/share/nltk_data'
    - '/home/vdeshpande/miniconda3/envs/general/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [4]:
# regexps for text preprocessing
ONLY_ALPHA = re.compile(r'([^\s\w]|_)+')
NUMBERS = re.compile(r'\b\d+')
MULTISPACE = re.compile(r'[^\S\r\n]{2,}')
AT_DIGIT = re.compile(r'@[,.]@')
AT_HYPHEN = re.compile(r'@-@')
WORD = re.compile(r'\w+')
HLINK = re.compile(r'http\S+')


def preprocess(line):
    #line = line.replace('``', '"')
    #line = line.replace("''", '"')
    #line = AT_HYPHEN.sub('-', line)
    line = line.replace('-', ' ')
    line = AT_DIGIT.sub('#NUMBER', line)
    line = NUMBERS.sub('#NUMBER ', line)
    line = HLINK.sub('#HLINK', line)
    #line = MULTISPACE.sub(' ', line)
    #line = line.lstrip(' ')
    return line

In [5]:
wiki_data = load_dataset("wikipedia", "20220301.en")
wiki_docs = wiki_data['train']['text']
wiki_vocab = pd.read_csv('wikipedia_vocab_regex_based.csv').iloc[:, 1].values.tolist()

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 6.31MB/s]


MissingBeamOptions: Trying to generate a dataset using Apache Beam, yet no Beam Runner or PipelineOptions() has been provided in `load_dataset` or in the builder arguments. For big datasets it has to run on large-scale data processing tools like Dataflow, Spark, etc. More information about Apache Beam runners at https://beam.apache.org/documentation/runners/capability-matrix/
If you really want to run it locally because you feel like the Dataset is small enough, you can use the local beam runner called `DirectRunner` (you may run out of memory). 
Example of usage: 
	`load_dataset('wikipedia', '20220301.en', beam_runner='DirectRunner')`

In [ ]:
english_vocab = [] + wiki_vocab
article_idx = 0
for article in tqdm(wiki_docs):
    article_idx += 1
    """
    if article_idx < 250000:
        continue
    """
    article = preprocess(article)
    english_vocab += WORD.findall(article.lower())#word_tokenize(article.lower())
        
    #
    if article_idx%10000 == 0:
        english_vocab = list(set(english_vocab))
        pd.DataFrame(english_vocab).to_csv('wikipedia_vocab_regex_based.csv')
        
        if len(english_vocab) >= 2e6:
            break
            

print(len(set(english_vocab)))

In [ ]:
english_vocab = [str(i) for i in english_vocab]

In [ ]:
missing_nltk = set(nltk_vocab) - set(english_vocab)
missing_aoc = set(aoc_vocab) - set(english_vocab)

In [ ]:
print('='*10)
print(f'number of words in NLTK but not in Wiki: {len(missing_nltk)}')
#print(missing_nltk)

print('='*10)
print(f'number of words in AOC but not in Wiki: {len(missing_aoc)}')
#print(missing_aoc)

In [ ]:
english_vocab += list(nltk_vocab)
english_vocab += list(aoc_vocab)
english_vocab = list(set(english_vocab))
pd.DataFrame(english_vocab).to_csv('wikipedia_vocab_regex_based.csv')

In [ ]:
random_articles = np.random.choice([i for i in range(260000, len(wiki_data['train']))], size=260000, replace=False)
new_articles = [wiki_docs[i] for i in random_articles]

In [ ]:
not_in_vocab = []
new_vocab = []
article_idx = 0
for article in tqdm(new_articles):
    article_idx += 1
    article = preprocess(article)
    new_vocab += WORD.findall(article.lower())#word_tokenize(article.lower())
        
    #
    if article_idx%50000 == 0:
        new_vocab = list(set(new_vocab))
        not_in_vocab += list(set(new_vocab) - set(english_vocab))
        not_in_vocab = list(set(not_in_vocab))
        print(f'There are {len(not_in_vocab)} number of words found that are not present in the vocabulary')
        
        if len(new_vocab) >= 2e6:
            break


In [ ]:
print(len(new_vocab))

In [ ]:
# merge both vocab
english_vocab += list(new_vocab)
english_vocab = list(set(english_vocab))
pd.DataFrame(english_vocab).to_csv('wikipedia_vocab_regex_based.csv')